In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/Users/danieloliveiradebrito/Projetos/cd4ml-sales-forecasting


/Users/danieloliveiradebrito/Projetos/cd4ml-sales-forecasting/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Mini EDA

In [3]:
import polars as pl

In [7]:
pl.scan_csv('data/train.csv').select(pl.col('store_nbr')).unique().collect()

store_nbr
i64
21
27
39
9
36
…
50
29
17


In [8]:
pl.scan_csv('data/train.csv').filter(pl.col('store_nbr') == 47).head().collect()

id,date,store_nbr,item_nbr,unit_sales,onpromotion
i64,str,i64,i64,f64,str
35958,"""2013-01-02""",47,103665,11.0,null
35959,"""2013-01-02""",47,105574,57.0,null
35960,"""2013-01-02""",47,105575,98.0,null
35961,"""2013-01-02""",47,105577,2.0,null
35962,"""2013-01-02""",47,105693,8.0,null


In [17]:
(pl.scan_csv('data/train.csv')
 .filter(pl.col('store_nbr') == 47)
 .with_columns(pl.col('date').str.to_date(format='%Y-%m-%d'))
 .with_columns(pl.col('date').dt.year().alias('year'))
 .group_by('year')
 .len()
 .collect())

year,len
i32,u32
2015,758018
2017,625988
2014,645746
2016,949683
2013,477972


In [26]:
store47_2016 = (
    pl.scan_csv('data/train.csv')
    .filter(pl.col('store_nbr') == 47)
    .with_columns(
        pl.col('date').str.to_date(format='%Y-%m-%d'),
        pl.col('onpromotion').replace_strict({"False": False, "True": True}))
    .filter(pl.col('date').dt.year().cast(pl.Int32) == 2016)
    .collect()
)

In [28]:
store47_2016.write_parquet('data/store47_2016.zstd', compression='zstd')

In [29]:
pl.scan_parquet('data/store47_2016.zstd').head().collect()

id,date,store_nbr,item_nbr,unit_sales,onpromotion
i64,date,i64,i64,f64,bool
66545998,2016-01-02,47,103520,10.0,true
66545999,2016-01-02,47,103665,7.0,false
66546000,2016-01-02,47,105574,33.0,false
66546001,2016-01-02,47,105575,80.0,false
66546002,2016-01-02,47,105577,13.0,false


In [31]:
pl.scan_parquet('data/store47_2016.zstd').describe()

statistic,id,date,store_nbr,item_nbr,unit_sales,onpromotion
str,f64,str,f64,f64,f64,f64
"""count""",949683.0,"""949683""",949683.0,949683.0,949683.0,949683.0
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0
"""mean""",8.4195e7,"""2016-07-04 18:42:04.154000""",47.0,1.1166e6,14.085485,0.09022
"""std""",1.0147e7,null,0.0,548828.00264,27.320898,null
"""min""",6.6545998e7,"""2016-01-02""",47.0,99197.0,-123.0,0.0
"""25%""",7.5359294e7,"""2016-04-04""",47.0,675524.0,3.0,null
"""50%""",8.4247521e7,"""2016-07-06""",47.0,1.157562e6,7.0,null
"""75%""",9.3018282e7,"""2016-10-05""",47.0,1.464177e6,15.0,null
"""max""",1.01674584e8,"""2016-12-31""",47.0,2.124052e6,5755.0,1.0


In [33]:
# remove negative unit_sales(
(pl.scan_parquet('data/store47_2016.zstd')
.filter(pl.col('unit_sales') >= 0)
.collect()
.write_parquet('data/store47_2016_positive.zstd', compression='zstd')
)

# Download

In [2]:
import polars as pl
import pyarrow.dataset as ds

dset = ds.dataset('s3://cd4ml/data/raw/store47_2016_positive.zstd', format='parquet')
(
    pl.scan_pyarrow_dataset(dset)
    .head()
    .collect()
)

id,date,store_nbr,item_nbr,unit_sales,onpromotion
i64,date,i64,i64,f64,bool
66545998,2016-01-02,47,103520,10.0,true
66545999,2016-01-02,47,103665,7.0,false
66546000,2016-01-02,47,105574,33.0,false
66546001,2016-01-02,47,105575,80.0,false
66546002,2016-01-02,47,105577,13.0,false
